In [3]:
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
from alpaca.data.historical import  StockHistoricalDataClient,NewsClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.common.enums import Sort
from datetime import datetime


load_dotenv()

api = os.getenv('ALPACA_API_KEY')
secret = os.getenv('ALPACA_SECRET_KEY')

def get_data(symbol ='AAPL', api = os.getenv('ALPACA_API_KEY') ,secret = os.getenv('ALPACA_SECRET_KEY')):
    # Get the current date and time
    now = datetime.now()

    # Format the datetime object into YYYYMMDD format
    formatted_date = now.strftime('%Y%m%d')

    # Convert the formatted string back into a datetime object
    formatted_datetime = datetime.strptime(formatted_date, '%Y%m%d')

    client = StockHistoricalDataClient(api, secret)
    params = StockBarsRequest(
        symbol_or_symbols=symbol,timeframe=TimeFrame.Day, start=datetime(2024,6, 1), end=formatted_datetime, limit=100,sort=Sort.DESC
        
    )

    data = client.get_stock_bars(params)
    data = dict(data)
    check = []
    for i in data['data'][symbol]:
        check.append(dict(i))
    df = pd.DataFrame(check)
    df.sort_values(by='timestamp',inplace=True)
    # Convert 'timestamp' to datetime and extract only the date
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date

    # Rename columns
    df.rename(columns={
        'timestamp': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close'
    }, inplace=True)

    # Keep only the required columns
    df = df[['Date', 'Open', 'High', 'Low', 'Close']]
    df.set_index('Date', inplace=True)

    return df

In [4]:
get_data()

,Open,High,Low,Close
Date,,,,
2024-11-22,228.060,230.7199,228.0600,229.87
2024-11-25,231.460,233.2450,229.7400,232.87
2024-11-26,233.330,235.5700,233.3300,235.06
2024-11-27,234.465,235.6900,233.8101,234.93
2024-11-29,234.805,237.8100,233.9700,237.33
...,...,...,...,...
2025-04-14,211.440,212.9400,201.1621,202.52
2025-04-15,201.855,203.5100,199.8000,202.14
2025-04-16,198.360,200.7000,192.3700,194.27
